In [47]:
import numpy as np 
import pandas as pd

In [48]:
matches=pd.read_csv('rfdataset1.csv')
matches.head()

,Start Date,Team,Opposition,Ground,Score,RPW,Result
0,13 March 2020,Australia,New Zealand,Sydney,258,5.16,Australia
1,7 Mar 2020,Australia,South Africa,Potchefstroom,254,5.08,South Africa
2,4 Mar 2020,Australia,South Africa,Bloemfontein,271,5.42,South Africa
3,1 Mar 2020,Sri Lanka,West Indies,Pallekele,307,6.14,Sri Lanka
4,29 Feb 2020,South Africa,Australia,Paarl,291,5.82,South Africa


In [36]:
matches.replace(['Australia','England','Pakistan','Sri Lanka','India','New Zealand',
                 'West Indies','South Africa']
                ,['AUS','ENG','PAK','SL','IND','NZ','WI','SA'],inplace=True)

matches.head()


,Start Date,Team,Opposition,Ground,Score,RPO,Result
0,13 March 2020,AUS,NZ,Sydney,258,5.16,AUS
1,7 Mar 2020,AUS,SA,Potchefstroom,254,5.08,SA
2,4 Mar 2020,AUS,SA,Bloemfontein,271,5.42,SA
3,1 Mar 2020,SL,WI,Pallekele,307,6.14,SL
4,29 Feb 2020,SA,AUS,Paarl,291,5.82,SA


In [37]:
TeamCode = {'Team': {'AUS':1,'ENG':2,'PAK':3,'SL':4,'IND':5,'NZ':6,'WI':7,'SA':8},
          'Opposition': {'AUS':1,'ENG':2,'PAK':3,'SL':4,'IND':5,'NZ':6,'WI':7,'SA':8},
          'Result': {'AUS':1,'ENG':2,'PAK':3,'SL':4,'IND':5,'NZ':6,'WI':7,'SA':8}}
matches.replace(TeamCode, inplace=True)
matches.head()


,Start Date,Team,Opposition,Ground,Score,RPO,Result
0,13 March 2020,1,6,Sydney,258,5.16,1
1,7 Mar 2020,1,8,Potchefstroom,254,5.08,8
2,4 Mar 2020,1,8,Bloemfontein,271,5.42,8
3,1 Mar 2020,4,7,Pallekele,307,6.14,4
4,29 Feb 2020,8,1,Paarl,291,5.82,8


In [38]:
#dicVal is a dictionary for teams
dicVal = TeamCode['Result']
print(dicVal['IND']) #print value from key
print(list(dicVal.keys())[list(dicVal.values()).index(7)])#find key by value
print(dicVal)


5
WI
{'AUS': 1, 'ENG': 2, 'PAK': 3, 'SL': 4, 'IND': 5, 'NZ': 6, 'WI': 7, 'SA': 8}


In [39]:
matches = matches[['Team','Opposition','Ground','Score','Result']]
#we tool only the columns which are relevant to the outcome i.e winner
matches.head()


,Team,Opposition,Ground,Score,Result
0,1,6,Sydney,258,1
1,1,8,Potchefstroom,254,8
2,1,8,Bloemfontein,271,8
3,4,7,Pallekele,307,4
4,8,1,Paarl,291,8


In [40]:
df = pd.DataFrame(matches)
df.describe()

,Team,Opposition,Score,Result
count,671.000000,671.000000,671.000000,671.000000
mean,4.254844,4.385991,261.360656,4.250373
std,2.295167,2.205498,63.653478,2.294685
min,1.000000,1.000000,67.000000,1.000000
25%,2.000000,3.000000,223.000000,2.000000
50%,4.000000,4.000000,266.000000,4.000000
75%,6.000000,6.000000,302.500000,6.000000
max,8.000000,8.000000,481.000000,8.000000


In [41]:
#toss won and matches won
temp=df['Result'].value_counts(sort=True)

print('\nNo of matches won by each team ')
for idx, val in temp.iteritems():
   print('{} -> {}'.format(list(dicVal.keys())[list(dicVal.values()).index(idx)],val))



No of matches won by each team 
IND -> 124
ENG -> 104
AUS -> 99
SA -> 93
SL -> 80
NZ -> 72
PAK -> 66
WI -> 33


In [42]:
df.dtypes

Team           int64
Opposition     int64
Ground        object
Score          int64
Result         int64
dtype: object

In [43]:
#since city,toss_decision and venue are object hence convert them to integers for modellng the data
from sklearn.preprocessing import LabelEncoder
var_mod = ['Ground']
le = LabelEncoder()
for i in var_mod:
    df[i] = le.fit_transform(df[i])
df.dtypes


Team          int64
Opposition    int64
Ground        int64
Score         int64
Result        int64
dtype: object

In [44]:
#building models to find the winner(Classification)-1)Logistic 2)RandomForest
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import metrics

#classification models and finding their performance:
def classification_model(model, data, predictors, outcome):
  model.fit(data[predictors],data[outcome])
  
  predictions = model.predict(data[predictors])
  
  accuracy = metrics.accuracy_score(predictions,data[outcome])
  print('Accuracy : %s' % '{0:.3%}'.format(accuracy))

In [45]:
#RandomForestClassifier

model = RandomForestClassifier(n_estimators=100)
outcome_var = ['Result']
predictor_var = ['Team', 'Opposition', 'Ground', 'Score']
classification_model(model, df,predictor_var,outcome_var)

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Accuracy : 99.851%


In [46]:
#Let's Play
team1='IND'
team2='AUS'
input=[dicVal[team1],dicVal[team2],'14','280']#team1,team2,venue,score
input = np.array(input).reshape((1, -1))
output=model.predict(input)
print(list(dicVal.keys())[list(dicVal.values()).index(output)])#finding key i.e team name from value

IND


In [20]:
#Let's Play
team1='ENG'
team2='AUS'
input=[dicVal[team1],dicVal[team2],'12','250']#team1,team2,venue,score
input = np.array(input).reshape((1, -1))
output=model.predict(input)
print(list(dicVal.keys())[list(dicVal.values()).index(output)])#finding key i.e team name from value

ENG


In [21]:
#Let's Play
team1='NZ'
team2='WI'
input=[dicVal[team1],dicVal[team2],'10','225']#team1,team2,venue,score
input = np.array(input).reshape((1, -1))
output=model.predict(input)
print(list(dicVal.keys())[list(dicVal.values()).index(output)])#finding key i.e team name from value

NZ
